# Camera Calibration with OpenCV

[This tutorial](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) is duplicated from offical tutorial of OpenCV, all materials mentioned in this tutorial can be obtained in [opencv github](https://github.com/opencv/opencv/tree/05b15943d6a42c99e5f921b7dbaa8323f3c042c6/samples/data)

### Goal

In this section, we will learn about

* types of distortion caused by cameras
* how to find the intrinsic and extrinsic properties of a camera
* how to undistort images based off these properties

### Basics

Some pinhole cameras introduce significant distortion to images. Two major kinds of distortion are radial distortion and tangential distortion.

Radial distortion causes straight lines to appear curved. Radial distortion becomes larger the farther points are from the center of the image. For example, one image is shown below in which two edges of a chess board are marked with red lines. But, you can see that the border of the chess board is not a straight line and doesn't match with the red line. All the expected straight lines are bulged out.

<img src="calib_radial.jpg" alt="My Image" width="350" height="450">

Radial distortion can be represented as follows:

$$
\begin{equation}
\begin{aligned}
x_{distored} &= x ( 1 + k_{1} r^{2} + k_{2} r^{4} + k_{3} r^{6} )\\
y_{distored} &= y ( 1 + k_{1} r^{2} + k_{2} r^{4} + k_{3} r^{6} )
\end{aligned}
\end{equation}
$$

Similarly, tangential distortion occurs because **the image-taking lense is not aligned perfectly parallel to the imaging plane**. So, some areas in the image may look nearer than expected. The amount of tangential distortion can be represented as below:

$$
\begin{equation}
\begin{aligned}
x_{distored} &= x + [2 p_{1} x y + p_{2} (r^{2} + 2 x^{2})] \\
y_{distored} &= y + [p_{1} (r^{2} + 2 y^{2}) + 2 p_{2} x y]
\end{aligned}
\end{equation}
$$

In short, we need to find five parameters, known as distortion coefficients given by:

$$
\begin{equation}
\begin{aligned}
Distortion \space Coefficients =(k_{1} \space k_{2} \space p_{1} \space p_{2} \space k_{3})
\end{aligned}
\end{equation}
$$


In addition to this, we need to some other information, like the **intrinsic** and **extrinsic** parameters of the camera. **Intrinsic parameters** are specific to a camera. They include information like focal length $(f_{x},f_{y})$ and optical centers $(c_{x},c_{y})$. The focal length and optical centers can be used to create a camera matrix, which can be used to remove distortion due to the lenses of a specific camera. The camera matrix is unique to a specific camera, so once calculated, it can be reused on other images taken by the same camera. It is expressed as a 3x3 matrix:


$$
\begin{equation}
\begin{aligned}
Camera \space Matrix = \left[\begin{matrix} f_{x} & 0 & c_{x} \cr 0 & f_{y} & c_{y} \cr 0 & 0 & 1 \end{matrix}\right]
\end{aligned}
\end{equation}
$$

For stereo applications, these distortions need to be corrected first. To find these parameters, we must provide some sample images of a well defined pattern (e.g. a chess board). We find some specific points of which we already know the relative positions (e.g. square corners in the chess board). We know the coordinates of these points in real world space and we know the coordinates in the image, so we can solve for the distortion coefficients. For better results, we need at least 10 test patterns.

### Code

As mentioned above, we need at least 10 test patterns for camera calibration. OpenCV comes with some images of a chess board (see samples/data/left01.jpg – left14.jpg), so we will utilize these. Consider an image of a chess board. The important input data needed for calibration of the camera is the set of 3D real world points and the corresponding 2D coordinates of these points in the image. 2D image points are OK which we can easily find from the image. (These image points are locations where two black squares touch each other in chess boards)

What about the 3D points from real world space? Those images are taken from a static camera and chess boards are placed at different locations and orientations. So we need to know $(X,Y,Z)$ values. But for simplicity, we can say chess board was kept stationary at XY plane, (so Z=0 always) and camera was moved accordingly. This consideration helps us to find only X,Y values. Now for X,Y values, we can simply pass the points as (0,0), (1,0), (2,0), ... which denotes the location of points. In this case, the results we get will be in the scale of size of chess board square. But if we know the square size, (say 30 mm), we can pass the values as (0,0), (30,0), (60,0), ... . Thus, we get the results in mm. (In this case, we don't know square size since we didn't take those images, so we pass in terms of square size).

3D points are called **object points** and 2D image points are called **image points**.

#### Setup

So to find pattern in chess board, we can use the function, cv.findChessboardCorners(). We also need to pass what kind of pattern we are looking for, like 8x8 grid, 5x5 grid etc. In this example, we use 7x6 grid. (Normally a chess board has 8x8 squares and 7x7 internal corners). It returns the corner points and retval which will be True if pattern is obtained. These corners will be placed in an order **(from left-to-right, top-to-bottom)**

#### Note

This function may not be able to find the required pattern in all the images. So, one good option is to write the code such that, it starts the camera and check each frame for required pattern. Once the pattern is obtained, find the corners and store it in a list. Also, provide some interval before reading next frame so that we can adjust our chess board in different direction. Continue this process until the required number of good patterns are obtained. Even in the example provided here, we are not sure how many images out of the 14 given are good. Thus, we must read all the images and take only the good ones.
Instead of chess board, we can alternatively use a circular grid. In this case, we must use the function cv.findCirclesGrid() to find the pattern. Fewer images are sufficient to perform camera calibration using a circular grid.

Once we find the corners, we can **increase their accuracy using cv.cornerSubPix()**. We can also draw the pattern using cv.drawChessboardCorners(). All these steps are included in below code:


In [1]:
import numpy as np
import cv2 as cv
import glob
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('samples/*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None) # ret: bool, if true: return value exists
    # If found, add object points, image points (after refining them)
    if ret == True:
        print(f'corners: {corners}')
        objpoints.append(objp)
        
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

corners: [[[300.66736  350.92874 ]]

 [[288.1165   325.6781  ]]

 [[275.791    298.81805 ]]

 [[261.79785  269.80862 ]]

 [[248.05641  239.62509 ]]

 [[233.0008   206.92723 ]]

 [[217.46024  172.23552 ]]

 [[331.7006   344.225   ]]

 [[321.02307  318.99838 ]]

 [[309.01877  290.95587 ]]

 [[296.43378  261.66263 ]]

 [[283.09103  230.15082 ]]

 [[268.70892  196.34387 ]]

 [[253.59319  160.58485 ]]

 [[365.26065  337.42664 ]]

 [[354.58194  311.00723 ]]

 [[343.8793   282.5702  ]]

 [[331.7146   252.3243  ]]

 [[319.64725  219.97707 ]]

 [[306.20053  185.59566 ]]

 [[292.4419   149.10506 ]]

 [[398.1472   329.42392 ]]

 [[389.0941   302.81833 ]]

 [[379.35495  273.79822 ]]

 [[368.87122  242.77551 ]]

 [[357.2025   209.89412 ]]

 [[344.77472  174.81343 ]]

 [[331.92804  137.13455 ]]

 [[432.3794   321.51923 ]]

 [[424.3185   294.14877 ]]

 [[415.52548  264.92874 ]]

 [[405.9928   233.22798 ]]

 [[395.82507  199.43097 ]]

 [[384.8827   163.29341 ]]

 [[372.54977  125.020744]]

 [[466.3452

corners: [[[254.72963  308.61407 ]]

 [[253.45633  280.52344 ]]

 [[252.63646  248.39021 ]]

 [[252.43504  213.82007 ]]

 [[251.49503  172.44621 ]]

 [[251.79442  128.6468  ]]

 [[251.45007   77.98987 ]]

 [[293.2903   318.045   ]]

 [[294.87354  290.14352 ]]

 [[297.06982  258.19284 ]]

 [[298.79138  222.08282 ]]

 [[301.44876  182.74187 ]]

 [[304.3669   135.65646 ]]

 [[306.8638    86.400604]]

 [[334.2001   327.5048  ]]

 [[338.1551   299.54575 ]]

 [[342.46857  267.66403 ]]

 [[347.64136  233.1402  ]]

 [[352.46246  192.01611 ]]

 [[357.4189   145.84972 ]]

 [[365.4095    96.584274]]

 [[374.9615   335.60226 ]]

 [[381.41925  308.43604 ]]

 [[388.7771   277.79404 ]]

 [[396.55096  241.73938 ]]

 [[404.76294  202.70882 ]]

 [[413.39362  158.33513 ]]

 [[425.5      106.5     ]]

 [[417.08984  343.6721  ]]

 [[425.3244   317.51547 ]]

 [[433.72485  287.03412 ]]

 [[445.25287  252.39241 ]]

 [[456.4296   213.6995  ]]

 [[469.12366  169.15547 ]]

 [[482.88855  120.33559 ]]

 [[457.0658

#### Calibration

Now that we have our object points and image points, we are ready to go for calibration. We can use the function, [cv.calibrateCamera()](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d) which returns the camera matrix, distortion coefficients, rotation and translation vectors etc.

In [2]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print(f'ret: {ret}')
print(f'camera matrix: {mtx}')
print(f'distortion coefficients: {dist}')
print(f'rotation vectors: {rvecs}')
print(f'translation vectors: {tvecs}')

ret: 0.15536906910078813
camera matrix: [[534.07088364   0.         341.53407553]
 [  0.         534.11914595 232.9456526 ]
 [  0.           0.           1.        ]]
distortion coefficients: [[-2.92971637e-01  1.07706962e-01  1.31038377e-03 -3.11018795e-05
   4.34798105e-02]]
rotation vectors: (array([[ 0.30697385],
       [ 0.5038552 ],
       [-1.82824733]]), array([[-0.45883216],
       [-0.08848877],
       [-1.33510786]]), array([[-0.45993978],
       [-0.3142018 ],
       [-1.76122223]]), array([[-0.43239599],
       [ 0.25603401],
       [-3.08832021]]), array([[-0.2645143 ],
       [-0.39360849],
       [-2.74787379]]), array([[-0.35367631],
       [-0.24363035],
       [-1.56874295]]), array([[-0.17288944],
       [-0.46764681],
       [ 1.34745198]]), array([[ 0.41531697],
       [ 0.65664497],
       [-1.3373494 ]]), array([[-0.37843358],
       [-0.18064237],
       [-3.11615996]]), array([[-0.29979221],
       [ 0.39216377],
       [-1.4348239 ]]), array([[-0.32034625],
 

#### Undistortion

Now, we can take an image and undistort it. OpenCV comes with two methods for doing this. However first, we can refine the camera matrix based on a free scaling parameter using [cv.getOptimalNewCameraMatrix()](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga7a6c4e032c97f03ba747966e6ad862b1). If the scaling parameter alpha=0, it returns undistorted image with minimum unwanted pixels. So it may even remove some pixels at image corners. If alpha=1, all pixels are retained with some extra black images. This function also returns an image ROI which can be used to crop the result.

In [3]:
img = cv.imread('samples/left12.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [6]:
# 1. Using cv.undistort()
# This is the easiest way. Just call the function and use ROI obtained above to crop the result.

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

cv.imshow('undistort image: dst', dst)
cv.waitKey(0)
cv.destroyAllWindows()

In [5]:
# 2. Using remapping
# This way is a little bit more difficult.
# First, find a mapping function from the distorted image to the undistorted image.
# Then use the remap function.

# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

cv.imshow('undistort image: dst', dst)
cv.waitKey(0)
cv.destroyAllWindows()

Still, both the methods give the same result. See the result below:

<img src="calib_result.jpg" alt="My Image" width="350" height="450">

You can see in the result that all the edges are straight.

Now you can store the camera matrix and distortion coefficients using write functions in NumPy (np.savez, np.savetxt etc) for future uses.

#### Re-projection Error

Re-projection error gives a good estimation of just how exact the found parameters are. The closer the re-projection error is to zero, the more accurate the parameters we found are. Given the intrinsic, distortion, rotation and translation matrices, we must first transform the object point to image point using [cv.projectPoints()](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga1019495a2c8d1743ed5cc23fa0daff8c). Then, we can calculate the absolute norm between what we got with our transformation and the corner finding algorithm. To find the average error, we calculate the arithmetical mean of the errors calculated for all the calibration images.

In [8]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.023686000375385673
